# 数据预处理

> 以下代码仅作为示例，请勿在生产环境直接使用。

推荐使用 [jupyter](https://jupyter.org/) 运行本教程。

隐语提供了多种预处理工具来处理数据。

## 1. 前置准备

初始化隐语，创建alice和bob两个参与方。

> 💡 在使用预处理之前，您可以需要先了解隐语 [DataFrame](https://www.secretflow.org.cn/zh-CN/docs/secretflow/v1.8.0b0/user_guide/preprocessing/DataFrame) 。

In [78]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob'], address='local')
alice = sf.PYU('alice')
bob = sf.PYU('bob')

The version of SecretFlow: 1.8.0b0


/home/beng003/anaconda/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-07-20 20:23:25,445	INFO worker.py:1724 -- Started a local Ray instance.


## 2. 数据准备

我们使用 [iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html) 作为示例数据集。

In [79]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)
data = pd.concat([iris.data, iris.target], axis=1)

# In order to facilitate the subsequent display,
# here we first set some data to None.
data.iloc[1, 1] = None
data.iloc[100, 1] = None

# Restore target to its original name.
data['target'] = data['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})
data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,NaN,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


Create a vertical partitioned DataFrame.

In [80]:
import tempfile
from secretflow.data.vertical import read_csv as v_read_csv

# Vertical partitioning.
v_alice, v_bob = data.iloc[:, :2], data.iloc[:, 2:]

# Save to temprary files.
_, alice_path = tempfile.mkstemp()
_, bob_path = tempfile.mkstemp()
v_alice.to_csv(alice_path, index=False)
v_bob.to_csv(bob_path, index=False)


df = v_read_csv({alice: alice_path, bob: bob_path})

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


You can also create a horizontal partitioned DataFrame, which works the same with vertical partitioning for subsequent steps.

In [4]:
# from secretflow.data.horizontal import read_csv as h_read_csv
# from secretflow.security.aggregation import PlainAggregator
# from secretflow.security.compare import PlainComparator

# # Horizontal partitioning.
# h_alice, h_bob = data.iloc[:70, :], data.iloc[70:, :]

# # Save to temorary files.
# _, h_alice_path = tempfile.mkstemp()
# _, h_bob_path = tempfile.mkstemp()
# h_alice.to_csv(h_alice_path, index=False)
# h_bob.to_csv(h_bob_path, index=False)

# df = h_read_csv(
#     {alice: h_alice_path, bob: h_bob_path},
#     aggregator=PlainAggregator(alice),
#     comparator=PlainComparator(alice),
# )

## 3. 预处理

隐语提供了缺失值填充、标准化、分类数据编码、离散化等多种预处理功能，其使用方式和sklearn的预处理一致。

### 3.1 缺失值填充

DataFrame提供了fillna方法，可以和pandas一样对缺失值进行填充。

In [81]:
# Before filling, the sepal width (cm) is missing in two positions.
df.count()['sepal width (cm)']

148

In [82]:
# Fill sepal width (cm) with 10.
df.fillna(value={'sepal width (cm)': 10}).count()['sepal width (cm)']

150

### 3.2 标准化

#### 3.2.1 将特征缩放到某个范围(归一化，最小-最大标准化)。

隐语提供了 MinMaxScaler 用于把特征缩放到最大和最小值之间。MinMaxScaler的输入和输出形式均为DataFrame。

下面是将 sepal length (cm) 缩放到[0, 1]范围的示例。

In [84]:
from secretflow.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_sepal_len = scaler.fit_transform(df['sepal length (cm)'])

print('Min: ', scaled_sepal_len.min())
print('Max: ', scaled_sepal_len.max())

Min:  sepal length (cm)    0.0
dtype: float64
Max:  sepal length (cm)    1.0
dtype: float64


#### 3.2.2 方差缩放(Z-score标准化)

隐语提供了 StandardScaler 进行方差缩放。StandardScaler的输入和输出行为均为DataFrame。

下面是一个将 sepal length (cm) 进行方差缩放的例子。

#### class secretflow.preprocessing.StandardScaler(with_mean=True, with_std=True)
##### 参数

- **`with_mean`** (默认值为 `True`)
  - **说明**：是否在缩放之前对数据进行中心化。设置为 `True` 时，数据会先减去均值。设置为 `False` 时，数据不会被中心化（即均值为零）。
- **`with_std`** (默认值为 `True`)
  - **说明**：是否将数据缩放到单位方差。设置为 `True` 时，会将数据的标准差缩放到 1。设置为 `False` 时，数据的标准差保持不变（即标准差为 1）。


In [86]:
from secretflow.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_sepal_len = scaler.fit_transform(df['sepal length (cm)'])

print('Min: ', scaled_sepal_len.min())
print('Max: ', scaled_sepal_len.max())

Min:  sepal length (cm)   -1.870024
dtype: float64
Max:  sepal length (cm)    2.492019
dtype: float64


In [87]:
print(f"substituted ds in alice:\n {sf.reveal(scaled_sepal_len.partitions[alice].data)}")

substituted ds in alice:
      sepal length (cm)
0            -0.900681
1            -1.143017
2            -1.385353
3            -1.506521
4            -1.021849
..                 ...
145           1.038005
146           0.553333
147           0.795669
148           0.432165
149           0.068662

[150 rows x 1 columns]


### 3.3 分类特征编码

#### 3.3.1 独热编码

隐语提供了 OneHotEncoder 用作独热编码。 OneHotEncoder的输入和输出行为均为DataFrame。

下面是一个对target列进行独热编码的例子。

#### class secretflow.preprocessing.OneHotEncoder (min_frequency=None, max_categories=None)

##### 参数

- `min_frequency`

  :

  - 类型：`int` 或 `float` 或 `None`
  - 说明：指定类别的最小频率，低于此频率的类别将被视为不常见。
    - 如果为 `int`：较少的类别数将被视为不常见。
    - 如果为 `float`：类别数小于 `min_frequency * n_samples` 的类别将被视为不常见。

- `max_categories`

  :

  - 类型：`int` 或 `None`
  - 说明：指定每个输入特征的输出特征的上限，考虑不常见类别时。包括不常见类别和常见类别的类别数。如果为 `None`，则没有限制。

In [89]:
from secretflow.preprocessing.encoder import OneHotEncoder
# from secretflow.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()
onehot_target = onehot_encoder.fit_transform(df['target'])

print('Columns: ', onehot_target.columns)
print('Min: \n', onehot_target.min())
print('Max: \n', onehot_target.max())

Columns:  ['target_setosa', 'target_versicolor', 'target_virginica']
Min: 
 target_setosa        0.0
target_versicolor    0.0
target_virginica     0.0
dtype: float64
Max: 
 target_setosa        1.0
target_versicolor    1.0
target_virginica     1.0
dtype: float64


#### 3.2.2 标签编码

隐语提供了 LabelEncoder 用作将标签列编码至[0, 类别数 - 1]。LabelEncoder的输入输出形式均为DataFrame。

下面是一个对target列进行标签编码的例子。

In [91]:
# from secretflow.preprocessing import LabelEncoder
from secretflow.preprocessing.encoder import LabelEncoder

label_encoder = LabelEncoder()
encoded_label = label_encoder.fit_transform(df['target'])

print('Columns: ', encoded_label.columns)
print('Min: \n', encoded_label.min())
print('Max: \n', encoded_label.max())

Columns:  ['target']
Min: 
 target    0
dtype: int64
Max: 
 target    2
dtype: int64


#### 3.2.3 序数编码(顺序编码)
VOrdinalEncoder 是一个用于将分类特征编码为序数（ordinal numbers）的类，类似于 sklearn.preprocessing.OrdinalEncoder，但其输入和输出是联邦数据框（federated dataframe）。当前仅支持垂直序数编码。

In [93]:
from secretflow.preprocessing.encoder import VOrdinalEncoder
oe = VOrdinalEncoder()
oe.fit(df['target'])
oe_df=oe.transform(df['target'])
print(f"substituted ds in bob:\n {sf.reveal(oe_df.partitions[bob].data)}")

substituted ds in bob:
      target
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
..      ...
145     2.0
146     2.0
147     2.0
148     2.0
149     2.0

[150 rows x 1 columns]


## 4 离散化

隐语提供了 KBinsDiscretizer 用作将连续数据切分成离散值。KBinsDiscretizer的输入输出形式均为DataFrame。

下面是一个将 petal length (cm) 切分成5个分桶的例子。

#### class secretflow.preprocessing.KBinsDiscretizer(n_bins=5, strategy: str = 'quantile')
##### 参数：

- `n_bins`：要生成的箱子数量。
- `strategy`：分箱策略，可以是 `'uniform'`（均匀分箱）或 `'quantile'`（分位数分箱），注意目前不支持 `'kmeans'`。

In [10]:
from secretflow.preprocessing import KBinsDiscretizer

estimator = KBinsDiscretizer(n_bins=5)
binned_petal_len = estimator.fit_transform(df['petal length (cm)'])

print('Min: \n', binned_petal_len.min())
print('Max: \n', binned_petal_len.max())

Min: 
 petal length (cm)    0.0
dtype: float64
Max: 
 petal length (cm)    4.0
dtype: float64


## 5 WOE编码

隐语提供了 VertWoeBinning ，可以对特征按照数量或者chimerge方法进行分桶，并计算每个分桶的WOE和IV值。**VertBinSubstitution** 可以把特征值替换成WOE值。

下面是一个对特征进行WOE编码的例子。

In [94]:
# woe binning use SPU or HEU device to protect label
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))

# Only support binary classification label dataset for now.
# use linear dataset as example
from secretflow.utils.simulation.datasets import load_linear

vdf = load_linear(parts={alice: (1, 4), bob: (18, 22)})
print(f"orig ds in alice:\n {sf.reveal(vdf.partitions[alice].data)}")
print(f"orig ds in bob:\n {sf.reveal(vdf.partitions[bob].data)}")

from secretflow.preprocessing.binning.vert_woe_binning import VertWoeBinning

binning = VertWoeBinning(spu)
bin_rules = binning.binning(
    vdf,
    binning_method="quantile",
    bin_num=5,
    bin_names={alice: ["x1", "x2", "x3"], bob: ["x18", "x19", "x20"]},
    label_name="y",
)

print(f"bin_rules for alice:\n {sf.reveal(bin_rules[alice])}")
print(f"bin_rules for bob:\n {sf.reveal(bin_rules[bob])}")

from secretflow.preprocessing.binning.vert_bin_substitution import VertBinSubstitution

woe_sub = VertBinSubstitution()
sub_data = woe_sub.substitution(vdf, bin_rules)

print(f"substituted ds in alice:\n {sf.reveal(sub_data[0].partitions[alice].data)}")
print(f"substituted ds in bob:\n {sf.reveal(sub_data[0].partitions[bob].data)}")

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorVertWoeBinningPyuWorker'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorVertWoeBinningPyuWorker'> with party bob.


orig ds in alice:
             x1        x2        x3
0    -0.514226  0.730010 -0.730391
1    -0.725537  0.482244 -0.823223
2     0.608353 -0.071102 -0.775098
3    -0.686642  0.160470  0.914477
4    -0.198111  0.212909  0.950474
...        ...       ...       ...
9995 -0.367246 -0.296454  0.558596
9996  0.010913  0.629268 -0.384093
9997 -0.238097  0.904069 -0.344859
9998  0.453686 -0.375173  0.899238
9999 -0.776015 -0.772112  0.012110

[10000 rows x 3 columns]
orig ds in bob:
            x18       x19       x20  y
0     0.810261  0.048303  0.937679  1
1     0.312728  0.526637  0.589773  1
2     0.039087 -0.753417  0.516735  0
3    -0.855979  0.250944  0.979465  1
4    -0.238805  0.243109 -0.121446  1
...        ...       ...       ... ..
9995 -0.847253  0.069960  0.786748  1
9996 -0.502486 -0.076290 -0.604832  1
9997 -0.424209  0.434947  0.998955  1
9998  0.914291 -0.473056  0.616257  1
9999 -0.602927 -0.021368  0.885519  0

[10000 rows x 4 columns]
bin_rules for alice:
 {'variables': 

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party alice.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


substituted ds in alice:
             x1        x2        x3
0     0.104363  0.550856 -0.537113
1     0.138189  0.320619 -0.537113
2    -0.160554  0.021751 -0.537113
3     0.138189  0.021751  0.630500
4     0.012474  0.320619  0.630500
...        ...       ...       ...
9995  0.104363 -0.178001  0.344572
9996  0.012474  0.550856 -0.257626
9997  0.104363  0.550856 -0.257626
9998 -0.083126 -0.178001  0.630500
9999  0.138189 -0.579551 -0.022037

[10000 rows x 3 columns]
substituted ds in bob:
            x18       x19       x20  y
0    -0.625846  0.061544 -0.908316  1
1    -0.385675  0.157737 -0.461821  1
2     0.097175 -0.242688 -0.461821  0
3     0.764487  0.157737 -0.908316  1
4     0.379689  0.157737  0.157737  1
...        ...       ...       ... ..
9995  0.764487  0.061544 -0.908316  1
9996  0.379689  0.061544  1.108304  1
9997  0.379689  0.157737 -0.908316  1
9998 -0.625846 -0.188862 -0.908316  1
9999  0.764487  0.061544 -0.908316  0

[10000 rows x 4 columns]


In [21]:
print(sub_data )

(VDataFrame(partitions={PYURuntime(alice): <secretflow.data.core.partition.Partition object at 0x7faac41a2680>, PYURuntime(bob): <secretflow.data.core.partition.Partition object at 0x7faaa0657f40>}, aligned=True), {'x20', 'x2', 'x3', 'x18', 'x1', 'x19'})


## 6 数据筛选

`secretflow.preprocessing.cond_filter_v` 模块提供了一些用于在数据表中应用条件过滤的工具。这个模块的核心类是 `ConditionFilter`，以及一些辅助函数。下面详细介绍这些功能及如何使用它们：

### `ConditionFilter` 类

#### 说明

`ConditionFilter` 类用于根据单个列的值和指定条件对表格进行过滤。它类似于 SQL 中的 `WHERE` 子句。

#### 构造函数

```
python复制代码class ConditionFilter(_PreprocessBase):
    def __init__(self, field_name: str, comparator: str, value_type: str, bound_value: List[str], float_epsilon: float = 1e-06):
        """
        初始化 ConditionFilter 实例。

        参数:
        - field_name: 列名，表示要应用过滤的列。
        - comparator: 比较运算符，决定了过滤条件（如 '>', '<', '==', '!=', '>=', '<='）。
        - value_type: 过滤值的类型（'FLOAT', 'STRING'(指字符串)）。
        - bound_value: 比较值，依据 comparator 进行过滤。
        - float_epsilon: 用于浮点数比较的容忍误差。
        """
        pass
```

#### 方法

1. **`fit(df: VDataFrame) → ConditionFilter`**
   - **功能**: 适配数据，通常用于计算需要的统计数据。
   - **参数**: `df` - 要应用的 `VDataFrame` 实例。
   - **返回**: 适配后的 `ConditionFilter` 实例。
2. **`transform(df: VDataFrame) → VDataFrame`**
   - **功能**: 根据指定条件过滤 `VDataFrame`。
   - **参数**: `df` - 要应用条件的 `VDataFrame` 实例。
   - **返回**: 过滤后的 `VDataFrame` 实例。
3. **`fit_transform(df: VDataFrame) → VDataFrame`**
   - **功能**: 同时进行适配和转换。
   - **参数**: `df` - 要应用条件的 `VDataFrame` 实例。
   - **返回**: 过滤后的 `VDataFrame` 实例。
4. **`get_else_table() → VDataFrame`**
   - **功能**: 获取未满足过滤条件的表格数据。
   - **返回**: 过滤条件未满足的数据的 `VDataFrame` 实例。
5. **`get_params() → Dict[str, Any]`**
   - **功能**: 获取 `ConditionFilter` 实例的参数。
   - **返回**: 参数字典。

##### ？？？
- secretflow.preprocessing.cond_filter_v.series_close_producer(epsilon=1e-06)
- secretflow.preprocessing.cond_filter_v.or_series(list_of_series)
- secretflow.preprocessing.cond_filter_v.series_float_isin_producer(epsilon=1e-06)

In [97]:
from secretflow.preprocessing.cond_filter_v import ConditionFilter

# 创建条件过滤器实例
condition_filter = ConditionFilter(
    field_name='target',
    comparator='==',
    value_type='STRING',
    bound_value=['setosa'],
    float_epsilon=1e-6
)

# 适配数据
condition_filter.fit(df['target'])

# 过滤数据
filtered_df = condition_filter.transform(df['target'])

# 获取未满足条件的数据
else_df = condition_filter.get_else_table()

INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.
INFO:root:Create proxy actor <class 'secretflow.device.proxy.ActorPartitionAgent'> with party bob.


In [105]:
print(f"substituted ds in bob:\n {sf.reveal(df['target'].partitions[bob].data)}")
print(f"substituted ds in bob:\n {sf.reveal(filtered_df.partitions[bob].data)}")
print(f"substituted ds in bob:\n {sf.reveal(else_df.partitions[bob].data)}")

substituted ds in bob:
         target
0       setosa
1       setosa
2       setosa
3       setosa
4       setosa
..         ...
145  virginica
146  virginica
147  virginica
148  virginica
149  virginica

[150 rows x 1 columns]
substituted ds in bob:
     target
0   setosa
1   setosa
2   setosa
3   setosa
4   setosa
5   setosa
6   setosa
7   setosa
8   setosa
9   setosa
10  setosa
11  setosa
12  setosa
13  setosa
14  setosa
15  setosa
16  setosa
17  setosa
18  setosa
19  setosa
20  setosa
21  setosa
22  setosa
23  setosa
24  setosa
25  setosa
26  setosa
27  setosa
28  setosa
29  setosa
30  setosa
31  setosa
32  setosa
33  setosa
34  setosa
35  setosa
36  setosa
37  setosa
38  setosa
39  setosa
40  setosa
41  setosa
42  setosa
43  setosa
44  setosa
45  setosa
46  setosa
47  setosa
48  setosa
49  setosa
substituted ds in bob:
          target
50   versicolor
51   versicolor
52   versicolor
53   versicolor
54   versicolor
..          ...
145   virginica
146   virginica
147   virginica
148

## 7 `class secretflow.preprocessing.LogroundTransformer(decimals: int = 6, bias: float = 0.5)` 

##### 用于计算 `round(log2(x + bias))`。

##### 参数

- **`decimals`** (默认值为 `6`)
  - **说明**：指定对数计算后的结果保留的小数位数。即结果将被四舍五入到这个位数。
- **`bias`** (默认值为 `0.5`)
  - **说明**：在计算对数之前加上的偏置值。这个偏置值可以帮助避免对数计算中的数学问题，如对数零值。

In [109]:
from secretflow.preprocessing import LogroundTransformer

# 初始化 LogroundTransformer
transformer = LogroundTransformer(decimals=2, bias=1.0)

# # 获取转换器参数(不能使用)
# params = transformer.get_params()
# print("Transformer parameters:", params)

# 拟合并转换数据
transformed_df = transformer.fit_transform(df['petal length (cm)'])

## Ending

In [12]:
# Clean up temporary files

import os

try:
    os.remove(alice_path)
    os.remove(bob_path)
except OSError:
    pass